In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
!git reset --hard fbe67e465375231474a2ad80a4389efc77ecff99

fatal: destination path 'yolov5' already exists and is not an empty directory.
/content/drive/MyDrive/Colab Notebooks/SADIA Track-1/yolov5
Updating files: 100% (111/111), done.
HEAD is now at fbe67e4 Fix `OMP_NUM_THREADS=1` for macOS (#8624)


In [ ]:
# install dependencies as necessary
%cd yolov5
!pip install -qr requirements.txt  # install dependencies (ignore errors)
import torch

from IPython.display import Image, clear_output  # to display images
from utils.downloads import attempt_download  # to download models/datasets

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

[Errno 2] No such file or directory: 'yolov5'
/content/drive/MyDrive/Colab Notebooks/SADIA Track-1/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 44.2 MB/s eta 0:00:00
Setup complete. Using torch 1.13.1+cu116 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15109MB, multi_processor_count=40)


In [ ]:
!python ./detect.py --source ../dataset/images/Test --weights '../model/best.pt' --conf 0.4 --save-txt 

detect: weights=['../model/best.pt'], source=../dataset/images/Test, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
fatal: cannot change to '/content/drive/MyDrive/Colab': No such file or directory
YOLOv5 🚀 2023-1-19 Python-3.8.10 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
custom_YOLOv5s summary: 232 layers, 7273488 parameters, 0 gradients, 16.8 GFLOPs
image 1/2092 /content/drive/MyDrive/Colab Notebooks/SADIA Track-1/dataset/images/Test/006e04a34f54d116f6aff2bb31cb320c.jpg: 384x640 Done. (0.014s)
image 2/2092 /content/drive/MyDrive/Colab Notebooks/SADIA Track-1/dataset/images/Test/006e70d6a51af3e670bbe712a83aa1c5.jpg: 384x640 2 GARBAGEs,

In [ ]:
mapClass = {
    0:[7, 'BAD_BILLBOARD'],
    1:[6, 'BAD_STREETLIGHT'],
    2:[5, 'BROKEN_SIGNAGE'],
    3:[9, 'CLUTTER_SIDEWALK'],
    4:[4, 'CONSTRUCTION_ROAD'],
    5:[1, 'FADED_SIGNAGE'],
    6:[3, 'GARBAGE'],
    7:[0, 'GRAFFITI'],
    8:[2, 'POTHOLES'],
    9:[8, 'SAND_ON_ROAD'],
    10:[10, 'UNKEPT_FACADE']
}

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import patches
import numpy as np
import glob

files = glob.glob('./runs/detect/exp3/labels/*')

dataset = []
for label in files:
  a_file = open(label, "r")
  image_path = label.split('/')[-1]

  data = []
  mul = np.array([1,1920,1080, 1920, 1080])
  for line in a_file:
    stripped_line = line.strip() #data in txt file
    line_list = stripped_line.split() #split data by space
    line_list = list(map(float, line_list)) #str -> float
    line_list = list(map(int, np.array(line_list)*mul)) #normalize and convert into int
    data.append(line_list)
  a_file.close()

  #list data contains data in (x_center, y_center, width, height)
  # Convert (x_center, y_center, width, height) --> (x_min, y_min, width, height)
  # adding className
  data_final = []
  for x in data:
    c,xmax,xmin,ymax,ymin = x[0]*1, (x[1]-(x[3]/2))+x[3]*1, (x[1]-(x[3]/2)) , (x[2]-(x[4]/2))+x[4]*1, (x[2]-(x[4]/2))  
    data_final.append([mapClass[c][0],image_path,mapClass[c][1],int(xmax//2),int(xmin//2),int(ymax//2),int(ymin//2)])

  dataset += data_final

In [ ]:
import pandas as pd

result = pd.DataFrame(dataset, columns = ['class', 'image_path', 'name', 'xmax', 'xmin', 'ymax', 'ymin'])
result.to_csv('../submission.csv', index=False)